# Natural-class preserving structures with networkx
Nick Danis / <nsdanis@wustl.edu> / www.nickdanis.com

Supplemental material for *Natural class-preserving transductions among phonological representations*, presented at the Workshop on Model-Theoretic Representations in Phonology at Stony Brook University, September 22-24, 2022.

In [2]:
import networkx as nx
import pydot
import re
from networkx.drawing.nx_pydot import graphviz_layout
from networkx.utils.misc import graphs_equal
import networkx.algorithms.isomorphism as iso
from itertools import chain, combinations
from collections import defaultdict

def powerset(iterable):
    # from https://stackoverflow.com/questions/1482308/how-to-get-all-subsets-of-a-set-powerset
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

def nxify(theory):
    nx_theory = dict()
    for segment, rep in theory.items():
        G = nx.DiGraph()
        raw_nodes = list(set(chain(*rep)))
        nodes = [(node, {'label' : re.sub(r'\d+','',node)}) for node in raw_nodes]
        G.add_nodes_from(nodes)
        G.add_edges_from(list(rep))
        nx_theory[segment] = G
    return nx_theory

def draw_phono(G):
    '''draws graph as a tree'''
    nx.draw_networkx(G,graphviz_layout(G,prog='dot'),node_color='white')

def part_of_class(seg, natclass):
    '''takes a segment and a factor
    returns true if the induced subgraph of the segment with the nodes of that factor
    is isomorphic to the natural class factor'''
#     sub = seg.remove_nodes_from(n for n in seg if n not in natclass.nodes())
#     return graphs_equal(sub,natclass)
#     return graphs_equal(nx.subgraph(seg.to_undirected(),natclass.nodes()),natclass.to_undirected())
#     return nx.is_isomorphic(nx.subgraph(seg.to_undirected(),natclass.nodes),natclass.to_undirected(),node_match=nm)
    nm = iso.categorical_node_match("label",'')
    for fac in get_factors(seg):
        if nx.is_isomorphic(fac,natclass,node_match=nm):
            return True
    return False


def get_factors(G):
    '''gets all connected subgraphs (factors) of some graph object'''
    subgraph_nodes = [sub for sub in powerset(G.nodes()) if len(sub)>0]
    subgraphs = []
    for nodes in subgraph_nodes:
        subgraph = nx.subgraph(G,nodes)
        if nx.is_connected(subgraph.to_undirected()):
            subgraphs.append(nx.subgraph(G,nodes))
    return subgraphs

def get_theory_factors(nx_theory):
    '''collects all (unique) factors across all segments in a theory'''
    all_factors = []
    unique_factors = []
    for seg in nx_theory.keys():
        for fac in get_factors(nx_theory[seg]):
            all_factors.append(fac)
    for fac in all_factors:
        if len(unique_factors) == 0:
            unique_factors.append(fac)
        else:
            equal_to = []
            for f in unique_factors:
                if graphs_equal(f,fac):
                    equal_to.append(f)
            if len(equal_to) == 0:
                unique_factors.append(fac)
    return unique_factors

def find_natural_classes(nx_theory):
    '''finds the segments that all share some factor in a theory'''
    natural_classes = defaultdict(list)
    for fac in get_theory_factors(nx_theory):
        for seg in nx_theory.keys():
            if part_of_class(nx_theory[seg],fac):
                natural_classes[fac].append(seg)
    return natural_classes

def make_nce_dict(nx_theory):
    nce_dict = defaultdict(list)
    natural_classes = find_natural_classes(nx_theory)
    for key, value in natural_classes.items():
        nce_dict[tuple(sorted(value))].append(key)
    return nce_dict

def compare_theories(nx_theory_a, nx_theory_b, verbose=False):
    nce_dict_a = make_nce_dict(nx_theory_a)
    nce_dict_b = make_nce_dict(nx_theory_b)
    print(f"Natural classes unique to theory a:")
    unique_a = nce_dict_a.keys() - nce_dict_b.keys()
    for nce in unique_a:
        print(nce)
        if verbose:
            print('Shared factors:')
            for fac in nce_dict_a[nce]:
                print(fac.nodes, fac.edges)
    print(f"Natural classes unique to theory b:")
    unique_b = nce_dict_b.keys() - nce_dict_a.keys()
    for nce in unique_b:
        print(nce)
        if verbose:
            print('Shared factors:')
            for fac in nce_dict_b[nce]:
                print(fac.nodes, fac.edges)
    print("Natural classes in common:")
    common = nce_dict_a.keys() & nce_dict_b.keys()
    for nce in common:
        print(nce)
    return nce_dict_a, nce_dict_b

## Oakden 2021

In [3]:

yip = {
    'L' : {('s','-u'), ('-u','l')},
    'H' : {('s','+u'), ('+u','h')},
    'M1' : {('s','-u'), ('-u','h')},
    'M2' : {('s','+u'), ('+u','l')},
    'HM' : {('s','+u'), ('+u','h'), ('+u','l')},
    'MH' : {('s','+u'), ('+u','l'), ('+u','h')},
    'ML' : {('s','-u'), ('-u','l'), ('-u','h')},
    'LM' : {('s','-u'), ('-u','l'), ('-u','h')}
}

bao = {
    'L' : {('s','T'), ('T','-u'), ('T','c'), ('c','l')},
    'H' : {('s','T'), ('T','+u'), ('T','c'), ('c','h')},
    'M1' : {('s','T'), ('T','-u'), ('T','c'), ('c','h')},
    'M2' : {('s','T'), ('T','+u'), ('T','c'), ('c','l')},
    'HM' : {('s','T'), ('T','+u'), ('T','c'), ('c','h'), ('c','l')},
    'MH' : {('s','T'), ('T','+u'), ('T','c'), ('c','h'), ('c','l')},
    'ML' : {('s','T'), ('T','-u'), ('T','c'), ('c','h'), ('c','l')},
    'LM' : {('s','T'), ('T','-u'), ('T','c'), ('c','h'), ('c','l')},
}

yip_nx = nxify(yip)
bao_nx = nxify(bao)

In [5]:
bao_nx['H'].factors

AttributeError: 'DiGraph' object has no attribute 'factors'

In [ ]:
nx.is_isomorphic(bao_nx['L'],bao_nx['H'])

In [ ]:
nce_yip, nce_bao = compare_theories(yip_nx, bao_nx,verbose=True)

In [ ]:
draw_phono(nce_bao[('HM', 'LM', 'MH', 'ML')][2])

In [ ]:
# print(nx.nx_pydot.write_dot(G,'test.gv'))

## Place, full specification, all combos

In [ ]:
unified_f = {
    'p' : {('rt','Cpl'), ('Cpl','+lab'), ('Cpl','-dors'), ('Cpl','-cor'), ('rt','Vpl'), ('Vpl','-lab'), ('Vpl','-dors'), ('Vpl','-cor')},
    't' : {('rt','Cpl'), ('Cpl','-lab'), ('Cpl','-dors'), ('Cpl','+cor'), ('rt','Vpl'), ('Vpl','-lab'), ('Vpl','-dors'), ('Vpl','-cor')},
    'k' : {('rt','Cpl'), ('Cpl','-lab'), ('Cpl','+dors'), ('Cpl','-cor'), ('rt','Vpl'), ('Vpl','-lab'), ('Vpl','-dors'), ('Vpl','-cor')},
    'pw' : {('rt','Cpl'), ('Cpl','+lab'), ('Cpl','-dors'), ('Cpl','-cor'), ('rt','Vpl'), ('Vpl','+lab'), ('Vpl','-dors'), ('Vpl','-cor')},
    'tw' : {('rt','Cpl'), ('Cpl','-lab'), ('Cpl','-dors'), ('Cpl','+cor'), ('rt','Vpl'), ('Vpl','+lab'), ('Vpl','-dors'), ('Vpl','-cor')},
    'kw' : {('rt','Cpl'), ('Cpl','-lab'), ('Cpl','+dors'), ('Cpl','-cor'), ('rt','Vpl'), ('Vpl','+lab'), ('Vpl','-dors'), ('Vpl','-cor')},
    'pj' : {('rt','Cpl'), ('Cpl','+lab'), ('Cpl','-dors'), ('Cpl','-cor'), ('rt','Vpl'), ('Vpl','-lab'), ('Vpl','-dors'), ('Vpl','+cor')},
    'tj' : {('rt','Cpl'), ('Cpl','-lab'), ('Cpl','-dors'), ('Cpl','+cor'), ('rt','Vpl'), ('Vpl','-lab'), ('Vpl','-dors'), ('Vpl','+cor')},
    'kj' : {('rt','Cpl'), ('Cpl','-lab'), ('Cpl','+dors'), ('Cpl','-cor'), ('rt','Vpl'), ('Vpl','-lab'), ('Vpl','-dors'), ('Vpl','+cor')},
    'pɣ' : {('rt','Cpl'), ('Cpl','+lab'), ('Cpl','-dors'), ('Cpl','-cor'), ('rt','Vpl'), ('Vpl','-lab'), ('Vpl','+dors'), ('Vpl','-cor')},
    'tɣ' : {('rt','Cpl'), ('Cpl','-lab'), ('Cpl','-dors'), ('Cpl','+cor'), ('rt','Vpl'), ('Vpl','-lab'), ('Vpl','+dors'), ('Vpl','-cor')},
    'kɣ' : {('rt','Cpl'), ('Cpl','-lab'), ('Cpl','+dors'), ('Cpl','-cor'), ('rt','Vpl'), ('Vpl','-lab'), ('Vpl','+dors'), ('Vpl','-cor')},
    'kp' : {('rt','Cpl'), ('Cpl','+lab'), ('Cpl','+dors'), ('Cpl','-cor'), ('rt','Vpl'), ('Vpl','-lab'), ('Vpl','-dors'), ('Vpl','-cor')},
    'tp' : {('rt','Cpl'), ('Cpl','+lab'), ('Cpl','-dors'), ('Cpl','+cor'), ('rt','Vpl'), ('Vpl','-lab'), ('Vpl','-dors'), ('Vpl','-cor')},
    'kt' : {('rt','Cpl'), ('Cpl','-lab'), ('Cpl','+dors'), ('Cpl','+cor'), ('rt','Vpl'), ('Vpl','-lab'), ('Vpl','-dors'), ('Vpl','-cor')},
    'y' : {('rt','Cpl'), ('Cpl','-lab'), ('Cpl','-dors'), ('Cpl','-cor'), ('rt','Vpl'), ('Vpl','+lab'), ('Vpl','-dors'), ('Vpl','+cor')}, 
    'ʉ' : {('rt','Cpl'), ('Cpl','-lab'), ('Cpl','-dors'), ('Cpl','-cor'), ('rt','Vpl'), ('Vpl','+lab'), ('Vpl','-dors'), ('Vpl','-cor')},
    'u' : {('rt','Cpl'), ('Cpl','-lab'), ('Cpl','-dors'), ('Cpl','-cor'), ('rt','Vpl'), ('Vpl','+lab'), ('Vpl','+dors'), ('Vpl','-cor')},
    'i' : {('rt','Cpl'), ('Cpl','-lab'), ('Cpl','-dors'), ('Cpl','-cor'), ('rt','Vpl'), ('Vpl','-lab'), ('Vpl','-dors'), ('Vpl','+cor')},
    'ɨ' : {('rt','Cpl'), ('Cpl','-lab'), ('Cpl','-dors'), ('Cpl','-cor'), ('rt','Vpl'), ('Vpl','-lab'), ('Vpl','-dors'), ('Vpl','-cor')},
    'ɯ' : {('rt','Cpl'), ('Cpl','-lab'), ('Cpl','-dors'), ('Cpl','-cor'), ('rt','Vpl'), ('Vpl','-lab'), ('Vpl','+dors'), ('Vpl','-cor')},
}

v_features_f = {
    'p' : {('rt','Pl'), ('Pl','+lab'), ('Pl','-dors'), ('Pl','-cor'), ('rt','-rnd'), ('rt','-bck'), ('rt','-frnt')},
    't' : {('rt','Pl'), ('Pl','-lab'), ('Pl','-dors'), ('Pl','+cor'), ('rt','-rnd'), ('rt','-bck'), ('rt','-frnt')},
    'k' : {('rt','Pl'), ('Pl','-lab'), ('Pl','+dors'), ('Pl','-cor'), ('rt','-rnd'), ('rt','-bck'), ('rt','-frnt')},
    'pw' : {('rt','Pl'), ('Pl','+lab'), ('Pl','-dors'), ('Pl','-cor'), ('rt','+rnd'), ('rt','-bck'), ('rt','-frnt')},
    'tw' : {('rt','Pl'), ('Pl','-lab'), ('Pl','-dors'), ('Pl','+cor'), ('rt','+rnd'), ('rt','-bck'), ('rt','-frnt')},
    'kw' : {('rt','Pl'), ('Pl','-lab'), ('Pl','+dors'), ('Pl','-cor'), ('rt','+rnd'), ('rt','-bck'), ('rt','-frnt')},
    'pj' : {('rt','Pl'), ('Pl','+lab'), ('Pl','-dors'), ('Pl','-cor'), ('rt','-rnd'), ('rt','-bck'), ('rt','+frnt')},
    'tj' : {('rt','Pl'), ('Pl','-lab'), ('Pl','-dors'), ('Pl','+cor'), ('rt','-rnd'), ('rt','-bck'), ('rt','+frnt')},
    'kj' : {('rt','Pl'), ('Pl','-lab'), ('Pl','+dors'), ('Pl','-cor'), ('rt','-rnd'), ('rt','-bck'), ('rt','+frnt')},
    'pɣ' : {('rt','Pl'), ('Pl','+lab'), ('Pl','-dors'), ('Pl','-cor'), ('rt','-rnd'), ('rt','+bck'), ('rt','-frnt')},
    'tɣ' : {('rt','Pl'), ('Pl','-lab'), ('Pl','-dors'), ('Pl','+cor'), ('rt','-rnd'), ('rt','+bck'), ('rt','-frnt')},
    'kɣ' : {('rt','Pl'), ('Pl','-lab'), ('Pl','+dors'), ('Pl','-cor'), ('rt','-rnd'), ('rt','+bck'), ('rt','-frnt')},
    'kp' : {('rt','Pl'), ('Pl','+lab'), ('Pl','+dors'), ('Pl','-cor'), ('rt','-rnd'), ('rt','-bck'), ('rt','-frnt')},
    'tp' : {('rt','Pl'), ('Pl','+lab'), ('Pl','-dors'), ('Pl','+cor'), ('rt','-rnd'), ('rt','-bck'), ('rt','-frnt')},
    'kt' : {('rt','Pl'), ('Pl','-lab'), ('Pl','+dors'), ('Pl','+cor'), ('rt','-rnd'), ('rt','-bck'), ('rt','-frnt')},
    'y' : {('rt','Pl'), ('Pl','-lab'), ('Pl','-dors'), ('Pl','-cor'), ('rt','+rnd'), ('rt','-bck'), ('rt','+frnt')}, 
    'ʉ' : {('rt','Pl'), ('Pl','-lab'), ('Pl','-dors'), ('Pl','-cor'), ('rt','+rnd'), ('rt','-bck'), ('rt','-frnt')},
    'u' : {('rt','Pl'), ('Pl','-lab'), ('Pl','-dors'), ('Pl','-cor'), ('rt','+rnd'), ('rt','+bck'), ('rt','-frnt')},
    'i' : {('rt','Pl'), ('Pl','-lab'), ('Pl','-dors'), ('Pl','-cor'), ('rt','-rnd'), ('rt','-bck'), ('rt','+frnt')},
    'ɨ' : {('rt','Pl'), ('Pl','-lab'), ('Pl','-dors'), ('Pl','-cor'), ('rt','-rnd'), ('rt','-bck'), ('rt','-frnt')},
    'ɯ' : {('rt','Pl'), ('Pl','-lab'), ('Pl','-dors'), ('Pl','-cor'), ('rt','-rnd'), ('rt','+bck'), ('rt','-frnt')},
}

In [ ]:
unified_nx = nxify(unified_f)
v_features_nx = nxify(v_features_f)

In [ ]:
nce_unified, nce_v_features = compare_theories(unified_nx, v_features_nx, verbose=True)

## 4 segment, full specification

In [ ]:
unified_s = {
    'p' : {('rt','Cpl'), ('Cpl','+lab'), ('Cpl','-dors'), ('Cpl','-cor'), ('rt','Vpl'), ('Vpl','-lab2'), ('Vpl','-dors2'), ('Vpl','-cor2')},
    't' : {('rt','Cpl'), ('Cpl','-lab'), ('Cpl','-dors'), ('Cpl','+cor'), ('rt','Vpl'), ('Vpl','-lab2'), ('Vpl','-dors2'), ('Vpl','-cor2')},
    'pw' : {('rt','Cpl'), ('Cpl','+lab'), ('Cpl','-dors'), ('Cpl','-cor'), ('rt','Vpl'), ('Vpl','+lab2'), ('Vpl','-dors2'), ('Vpl','-cor2')},
    'u' : {('rt','Cpl'), ('Cpl','-lab'), ('Cpl','-dors'), ('Cpl','-cor'), ('rt','Vpl'), ('Vpl','+lab2'), ('Vpl','+dors2'), ('Vpl','-cor2')},
}

v_features_s = {
    'p' : {('rt','Pl'), ('Pl','+lab'), ('Pl','-dors'), ('Pl','-cor'), ('rt','-rnd'), ('rt','-bck'), ('rt','-frnt')},
    't' : {('rt','Pl'), ('Pl','-lab'), ('Pl','-dors'), ('Pl','+cor'), ('rt','-rnd'), ('rt','-bck'), ('rt','-frnt')},
    'pw' : {('rt','Pl'), ('Pl','+lab'), ('Pl','-dors'), ('Pl','-cor'), ('rt','+rnd'), ('rt','-bck'), ('rt','-frnt')},
    'u' : {('rt','Pl'), ('Pl','-lab'), ('Pl','-dors'), ('Pl','-cor'), ('rt','+rnd'), ('rt','+bck'), ('rt','-frnt')},
}

In [ ]:
unified_s_nx = nxify(unified_s)
v_features_s_nx = nxify(v_features_s)

In [ ]:
nce_unified_f, nce_v_features_f = compare_theories(unified_s_nx,v_features_s_nx,verbose=True)

In [ ]:
draw_phono(nce_v_features_f[('p', 'pw', 'u')][6])

### no unique nodes

In [ ]:
unified_s = {
    'p' : {('rt','Cpl'), ('Cpl','+lab'), ('Cpl','-dors'), ('Cpl','-cor'), ('rt','Vpl'), ('Vpl','-lab'), ('Vpl','-dors'), ('Vpl','-cor')},
    't' : {('rt','Cpl'), ('Cpl','-lab'), ('Cpl','-dors'), ('Cpl','+cor'), ('rt','Vpl'), ('Vpl','-lab'), ('Vpl','-dors'), ('Vpl','-cor')},
    'pw' : {('rt','Cpl'), ('Cpl','+lab'), ('Cpl','-dors'), ('Cpl','-cor'), ('rt','Vpl'), ('Vpl','+lab'), ('Vpl','-dors'), ('Vpl','-cor')},
    'u' : {('rt','Cpl'), ('Cpl','-lab'), ('Cpl','-dors'), ('Cpl','-cor'), ('rt','Vpl'), ('Vpl','+lab'), ('Vpl','+dors'), ('Vpl','-cor')},
}

v_features_s = {
    'p' : {('rt','Pl'), ('Pl','+lab'), ('Pl','-dors'), ('Pl','-cor'), ('rt','-rnd'), ('rt','-bck'), ('rt','-frnt')},
    't' : {('rt','Pl'), ('Pl','-lab'), ('Pl','-dors'), ('Pl','+cor'), ('rt','-rnd'), ('rt','-bck'), ('rt','-frnt')},
    'pw' : {('rt','Pl'), ('Pl','+lab'), ('Pl','-dors'), ('Pl','-cor'), ('rt','+rnd'), ('rt','-bck'), ('rt','-frnt')},
    'u' : {('rt','Pl'), ('Pl','-lab'), ('Pl','-dors'), ('Pl','-cor'), ('rt','+rnd'), ('rt','+bck'), ('rt','-frnt')},
}

In [ ]:
unified_s_nx = nxify(unified_s)
v_features_s_nx = nxify(v_features_s)

In [ ]:
nce_unified_d, nce_v_features_d = compare_theories(unified_s_nx,v_features_s_nx,verbose=True)

relabel nodes? https://networkx.org/documentation/stable/reference/generated/networkx.relabel.relabel_nodes.html

In [ ]:
unified_p = {
    'p' : {('rt','Cpl'), ('Cpl','lab')},
    't' : {('rt','Cpl'), ('Cpl','cor')},
    'u' : {('rt','Vpl'), ('Vpl','lab')},
    'pw' : {('rt','Cpl'), ('Cpl','lab'), ('rt','Vpl'), ('Vpl','lab')},
    
}

v_features_p = {
    'p' : {('rt','Pl'), ('Pl','lab'), ('rt','-rnd')},
    't' : {('rt','Pl'), ('Pl','cor'), ('rt','-rnd')},
    'pw' : {('rt','Pl'), ('Pl','lab'), ('rt','+rnd')},
    'u' : {('rt','+rnd')},
}

unified_p_nx = nxify(unified_p)
v_features_p_nx = nxify(v_features_p)

nce_unified, nce_v = compare_theories(unified_p_nx, v_features_p_nx, verbose=True)

In [ ]:
draw_phono(nce_unified[('p', 'pw', 'u')][0])

In [ ]:
draw_phono(nce_v[('p', 't')][1])